In [0]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

from keras.layers.core import Dense, Activation

import re
import csv
from string import punctuation
import nltk 
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
from nltk import word_tokenize
from nltk.util import ngrams
from nltk.corpus import stopwords
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score
import pickle
import numpy as np
import pandas as pd
                                                 
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score


Using TensorFlow backend.


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [0]:

#RÉCUPÉRATION DES TAGS
f=open("/data_task3/train.txt", "r")
data=[]
contents =[line for line in f.readlines() if line.strip()]
indx=0
for x in contents:
    words=x.split()
    if words[0]=="SAMPLE_START":
         indx+=1
    data.append([indx,words[0],words[1]])
data= pd.DataFrame(data, columns=['sentence', 'word', 'tag'])
print (data.head(60))

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive
 02Oct.png
 0x52D91BAC.asc
 1508.01991.pdf
 16_Invitation_Ndiaye.jpg
 1707.06799.pdf
 2016-09-27-PHOTO-00000001.jpg
 2016-09-29_Rechnung_Kundennr_108298610.pdf
 25-1.png
 25-2.png
 25-3.png
 25-4.png
 25-5.png
 25-6.png
 25-7.png
 25-9.png
 26-1.png
 78106SXWMZ720886181180525111238676411941.pdf
'Absolventen Forum 2019 Anmeldeformular.pdf'
'Analyse et fouille de donnees'
 ART_02-ROUTAGE.pdf
 Berichtsbogen_zum_Auslandsstudium1.pdf
 botnet1.png
'Carte sans titre 

In [0]:
words = list(set(data["word"].values))
words.append("ENDPAD")
n_words = len(words); n_words

tags = list(set(data["tag"].values))
n_tags = len(tags); n_tags

print(words)
print(tags)

['junction', 'Commissioner', 'Bishop', '1999', 'diamond', 'quiet', 'times', 'SAMPLE_START', 'kidnapped', 'else', 'Greater', 'Addressing', 'spattered', 'K.H.', 'Ganjam', 'responsibility', 'condemned', 'Gowtham', 'labourer', 'hotheads', 'half', 'Sangareddy', 'victim', 'railway', 'back-stabber', 'Sabha', 'City', 'business', 'Sub-Inspector', 'Slamming', 'Sardar', 'WARANGAL', 'towards', 'House', 'dragged', 'Harish', 'petrol', 'naxal', 'remaining', 'Action', 'himself', 'removal', 'Katchi', 'deliberate', 'gallery', '65', '4', 'rejected', 'someone', 'Week', 'godown', 'km', 'attempted', '“', 'issue', 'Only', 'got', 'Popular', 'Collector', 'conspiracy', 'Karunanidhi', '00:16', 'allegiance', 'Leader', 'Later', 'stalled', 'say', 'general', 'CAIT', 'learning', 'Kovilur', 'abduction', 'salute', 'discussion', 'Cornered', 'jawans', 'temporarily', 'effect', 'Marxist', 'Nara', 'slap', 'Alagiri', 'non-payment', 'motion', 'pays', 'Guru', 'occurred', 'college', 'steps', 'highways', 'Basia', 'demands', 'hib

In [0]:
class SentenceGetter(object):
    
    def __init__(self, dataset):
        self.n_sent = 1
        self.dataset = dataset
        self.empty = False
        agg_func = lambda s: [(w, t) for w, t in zip(s["word"].values.tolist(),
                                                       
                                                        s["tag"].values.tolist())]
        self.grouped = self.dataset.groupby("sentence").apply(agg_func)
        self.sentences = [s for s in self.grouped]
    
    def get_next(self):
        try:
            s = self.grouped["sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None
getter = SentenceGetter(data)
sentences = getter.sentences    
print(sentences)

[[('SAMPLE_START', 'O'), ('KRISHNAGIRI', 'O'), ('/', 'O'), ('DHARMAPURI', 'O'), (':', 'O'), ('Water', 'O'), ('supply', 'O'), ('disrupted', 'O'), (',', 'O'), ('villagers', 'B-participant'), ('block', 'B-trigger'), ('road', 'B-loc'), ('September', 'O'), ('04', 'O'), (',', 'O'), ('2014', 'O'), ('00:00', 'O'), ('IST', 'O'), ('Officials', 'O'), ('promise', 'O'), ('to', 'O'), ('supply', 'O'), ('water', 'O'), ('in', 'O'), ('tankers', 'O'), ('Women', 'B-participant'), ('took', 'B-trigger'), ('to', 'I-trigger'), ('streets', 'I-trigger'), ('carrying', 'O'), ('pots', 'O'), ('in', 'O'), ('protest', 'O'), ('against', 'O'), ('erratic', 'O'), ('supply', 'O'), ('of', 'O'), ('drinking', 'O'), ('water', 'O'), ('at', 'O'), ('Kelamagalam', 'B-place'), ('in', 'O'), ('Udhanapalli', 'B-place'), ('on', 'B-etime'), ('Wednesday', 'I-etime'), ('.', 'O'), ('[SEP]', 'O'), ('The', 'O'), ('protesters', 'B-participant'), ('blocked', 'B-trigger'), ('the', 'O'), ('Rayakottai', 'B-fname'), ('road', 'I-fname'), ('.', 'O'

In [0]:
import pickle
with open("/data_task3/list_tag_ner", 'rb') as v:
    lpostag = pickle.load( v)
print(lpostag)    
print(len(lpostag)   )


[[('SAMPLE_START', 'NNP'), ('KRISHNAGIRI', 'NNP'), ('/', 'NNP'), ('DHARMAPURI', 'NNP'), (':', ':'), ('Water', 'NNP'), ('supply', 'NN'), ('disrupted', 'VBD'), (',', ','), ('villagers', 'NNS'), ('block', 'VBP'), ('road', 'NN'), ('September', 'NNP'), ('04', 'CD'), (',', ','), ('2014', 'CD'), ('00:00', 'CD'), ('IST', 'NNP'), ('Officials', 'NNP'), ('promise', 'NN'), ('to', 'TO'), ('supply', 'VB'), ('water', 'NN'), ('in', 'IN'), ('tankers', 'NNS'), ('Women', 'NNP'), ('took', 'VBD'), ('to', 'TO'), ('streets', 'VB'), ('carrying', 'VBG'), ('pots', 'NNS'), ('in', 'IN'), ('protest', 'NN'), ('against', 'IN'), ('erratic', 'JJ'), ('supply', 'NN'), ('of', 'IN'), ('drinking', 'NN'), ('water', 'NN'), ('at', 'IN'), ('Kelamagalam', 'NNP'), ('in', 'IN'), ('Udhanapalli', 'NNP'), ('on', 'IN'), ('Wednesday', 'NNP'), ('.', '.'), ('[SEP]', 'VB'), ('The', 'DT'), ('protesters', 'NNS'), ('blocked', 'VBD'), ('the', 'DT'), ('Rayakottai', 'NNP'), ('road', 'NN'), ('.', '.')], [('SAMPLE_START', 'NNP'), ('KARNATAKA', '

In [0]:
sentences_postag=[[tuplesent + (tuplepos[1],)  for tuplesent,tuplepos in zip(i,j)]for i,j in zip(sentences,lpostag)]
print(sentences_postag)   
print(len(sentences_postag)   )

[[('SAMPLE_START', 'O', 'NNP'), ('KRISHNAGIRI', 'O', 'NNP'), ('/', 'O', 'NNP'), ('DHARMAPURI', 'O', 'NNP'), (':', 'O', ':'), ('Water', 'O', 'NNP'), ('supply', 'O', 'NN'), ('disrupted', 'O', 'VBD'), (',', 'O', ','), ('villagers', 'B-participant', 'NNS'), ('block', 'B-trigger', 'VBP'), ('road', 'B-loc', 'NN'), ('September', 'O', 'NNP'), ('04', 'O', 'CD'), (',', 'O', ','), ('2014', 'O', 'CD'), ('00:00', 'O', 'CD'), ('IST', 'O', 'NNP'), ('Officials', 'O', 'NNP'), ('promise', 'O', 'NN'), ('to', 'O', 'TO'), ('supply', 'O', 'VB'), ('water', 'O', 'NN'), ('in', 'O', 'IN'), ('tankers', 'O', 'NNS'), ('Women', 'B-participant', 'NNP'), ('took', 'B-trigger', 'VBD'), ('to', 'I-trigger', 'TO'), ('streets', 'I-trigger', 'VB'), ('carrying', 'O', 'VBG'), ('pots', 'O', 'NNS'), ('in', 'O', 'IN'), ('protest', 'O', 'NN'), ('against', 'O', 'IN'), ('erratic', 'O', 'JJ'), ('supply', 'O', 'NN'), ('of', 'O', 'IN'), ('drinking', 'O', 'NN'), ('water', 'O', 'NN'), ('at', 'O', 'IN'), ('Kelamagalam', 'B-place', 'NNP')

In [0]:
import pickle
with open("/data_task3/Icews_actors", 'rb') as f:
    Icews_actors = pickle.load( f)
print(Icews_actors)  

['Archbishop', 'Military Personnel Special', 'Deputy Chief Rachpal Singh', 'Hindustan Times', 'D S Mangat', 'Privitization Commission', 'Ns Maghu', 'S A Dange', 'Minist Of The Environment', 'R S Gavai', 'Shaukat H Guru', 'Ravi Shankar Prasad', 'Jasbir Sandu', 'Minister Shankarrao Chavan', 'Minist For The Attorney General', 'Susashi Tiwari', 'Surinder Chauhan', 'Nishant Maghu', 'Hari Singh', 'Meenakshi Lekhi', 'Ram Nath Kovind', 'S S Babbar', 'Chief Engineer', 'Indian Left Party', 'Prince', 'Diocese', 'Khalistan Liberation Force', 'Aide De Camp', 'Business Men', 'Kuki Militants', 'MCC', 'Governor Vora', 'G S Mann', 'Business Person', 'Worker Organisation', 'L P Barbosa', 'Maj S Oboke', 'Au Azimi', 'National Education Committee', 'Kiran Kumar Allam', 'M M Joshi', 'Nirmala Sitharaman', 'Liu Kun', 'Peoples Labor Party', 'Ombuds', 'Junta Forces', 'Gurka National Liberation Front', 'Mm Mittal', 'Indian Mujahidin', 'Christian Monk', 'Devyani Khobragade', 'Sailen Dasgupta', 'People For Animals

In [0]:
sentences_postag_dict=[[tuplesent + (1,) if tuplesent[0] in Icews_actors else tuplesent + (0,) for tuplesent in sent] for sent in sentences_postag]
print(sentences_postag_dict)

[[('SAMPLE_START', 'O', 'NNP', 0), ('KRISHNAGIRI', 'O', 'NNP', 0), ('/', 'O', 'NNP', 0), ('DHARMAPURI', 'O', 'NNP', 0), (':', 'O', ':', 0), ('Water', 'O', 'NNP', 0), ('supply', 'O', 'NN', 0), ('disrupted', 'O', 'VBD', 0), (',', 'O', ',', 0), ('villagers', 'B-participant', 'NNS', 0), ('block', 'B-trigger', 'VBP', 0), ('road', 'B-loc', 'NN', 0), ('September', 'O', 'NNP', 0), ('04', 'O', 'CD', 0), (',', 'O', ',', 0), ('2014', 'O', 'CD', 0), ('00:00', 'O', 'CD', 0), ('IST', 'O', 'NNP', 0), ('Officials', 'O', 'NNP', 0), ('promise', 'O', 'NN', 0), ('to', 'O', 'TO', 0), ('supply', 'O', 'VB', 0), ('water', 'O', 'NN', 0), ('in', 'O', 'IN', 0), ('tankers', 'O', 'NNS', 0), ('Women', 'B-participant', 'NNP', 1), ('took', 'B-trigger', 'VBD', 0), ('to', 'I-trigger', 'TO', 0), ('streets', 'I-trigger', 'VB', 0), ('carrying', 'O', 'VBG', 0), ('pots', 'O', 'NNS', 0), ('in', 'O', 'IN', 0), ('protest', 'O', 'NN', 0), ('against', 'O', 'IN', 0), ('erratic', 'O', 'JJ', 0), ('supply', 'O', 'NN', 0), ('of', 'O'

In [0]:
Icews_actors1 = [k.lower() for k in Icews_actors]
for ind,i in enumerate(sentences_postag):
  for j in i:
    if j[0] in Icews_actors:
      print(ind)
      print (j)

0
('Women', 'B-participant', 'NNP')
1
('Correspondent', 'O', 'NNP')
1
('Union', 'B-target', 'NNP')
1
('Government', 'I-target', 'NNP')
2
('BJP', 'B-organizer', 'NNP')
3
('Congress', 'B-organizer', 'NNP')
3
('Congress', 'O', 'NNP')
3
('People', 'O', 'NNPS')
3
('Congress', 'B-organizer', 'NNP')
3
('MP', 'O', 'NNP')
5
('Congress', 'O', 'NNP')
5
('Congress', 'I-organizer', 'NNP')
10
('Police', 'O', 'NNP')
13
('Congress', 'B-target', 'NNP')
13
('Congress', 'B-target', 'NNP')
13
('Congress', 'B-target', 'NNP')
14
('BJP', 'O', 'NNP')
14
('BJP', 'O', 'NNP')
14
('BJP', 'O', 'NNP')
14
('Kashmir', 'I-place', 'NNP')
16
('Kashmir', 'B-place', 'NNP')
16
('Kashmir', 'I-place', 'NNP')
16
('Christian', 'B-loc', 'JJ')
18
('Police', 'O', 'NNP')
18
('Transport', 'O', 'NNP')
18
('Corporation', 'O', 'NNP')
18
('Government', 'O', 'NNP')
20
('BJP', 'O', 'NNP')
20
('Congress', 'O', 'NNP')
21
('Representatives', 'B-participant', 'NNS')
23
('Union', 'I-organizer', 'NNP')
25
('Mukherjee', 'O', 'NNP')
26
('Muslim'

In [0]:
def isinDictionary(w,pos):
  t=""
 
  if (pos== "B-fname"  or"I-participant") and w in Icews_actors:
      t= 1
  else:
      t=0
  return(t)
  

In [0]:
def word2features(sent, i):
    word = sent[i][0]
    postag =sent[i][2]
    dict=sent[i][3]
    
    features = {
        'bias': 1.0, 
        'word.lower()': word.lower(), 
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'dict':isinDictionary(word,postag),
        #'postag[:1]': postag[:1],
        #'postag[:-1]': postag[:-1],
        #'postag[:-2]': postag[:-2],
        #'postag[:-3]': postag[:-3],
        #'dict[-1:]':isinDictionary(word[-1:],postag[:-1]),
        #'dict[-2:]':isinDictionary(word[-2:],postag[:-2]),
        #'dict[-3:]':isinDictionary(word[-3:],postag[:-3]),
        #'dict[1:]':isinDictionary(word[-1:],postag[:1]),
   
    }
    

    return features

def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, label,pos,dict in sent]

def sent2tokens(sent):
    return [token for token,  label,pos,dict in sent]

In [0]:
! pip install sklearn_crfsuite

In [0]:
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
from collections import Counter


In [0]:
from sklearn.model_selection import train_test_split
X = [sent2features(s) for s in sentences_postag_dict]
y = [sent2labels(s) for s in sentences_postag_dict]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=0)
print(X_train[0])

[{'bias': 1.0, 'word.lower()': 'sample_start', 'word[-3:]': 'ART', 'word[-2:]': 'RT', 'word.isupper()': True, 'word.istitle()': False, 'word.isdigit()': False, 'postag': 'NNP', 'dict': 0}, {'bias': 1.0, 'word.lower()': "'", 'word[-3:]': "'", 'word[-2:]': "'", 'word.isupper()': False, 'word.istitle()': False, 'word.isdigit()': False, 'postag': 'POS', 'dict': 0}, {'bias': 1.0, 'word.lower()': 'ksrtc', 'word[-3:]': 'RTC', 'word[-2:]': 'TC', 'word.isupper()': True, 'word.istitle()': False, 'word.isdigit()': False, 'postag': 'NNP', 'dict': 0}, {'bias': 1.0, 'word.lower()': 'suffering', 'word[-3:]': 'ing', 'word[-2:]': 'ng', 'word.isupper()': False, 'word.istitle()': False, 'word.isdigit()': False, 'postag': 'VBG', 'dict': 0}, {'bias': 1.0, 'word.lower()': 'from', 'word[-3:]': 'rom', 'word[-2:]': 'om', 'word.isupper()': False, 'word.istitle()': False, 'word.isdigit()': False, 'postag': 'IN', 'dict': 0}, {'bias': 1.0, 'word.lower()': 'operational', 'word[-3:]': 'nal', 'word[-2:]': 'al', 'word

In [0]:
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=200,
    all_possible_transitions=True
)
crf.fit(X_train, y_train)

CRF(algorithm='lbfgs', all_possible_states=None, all_possible_transitions=True,
    averaging=None, c=None, c1=0.1, c2=0.1, calibration_candidates=None,
    calibration_eta=None, calibration_max_trials=None, calibration_rate=None,
    calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
    gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=200,
    max_linesearch=None, min_freq=None, model_filename=None, num_memories=None,
    pa_type=None, period=None, trainer_cls=None, variance=None, verbose=False)

In [0]:
y_pred = crf.predict(X_test)
print(metrics.flat_classification_report(y_test, y_pred))

               precision    recall  f1-score   support

      B-etime       0.67      0.51      0.58        81
      B-fname       0.40      0.24      0.30        34
        B-loc       0.36      0.08      0.13        50
  B-organizer       0.63      0.37      0.46       153
B-participant       0.64      0.50      0.56       225
      B-place       0.54      0.26      0.35       116
     B-target       0.29      0.09      0.13       105
    B-trigger       0.83      0.59      0.69       309
      I-etime       0.70      0.53      0.60        96
      I-fname       0.27      0.26      0.26        86
        I-loc       0.37      0.17      0.23        82
  I-organizer       0.51      0.54      0.52       147
I-participant       0.07      0.03      0.04        62
      I-place       0.50      0.27      0.35        15
     I-target       0.32      0.22      0.26        50
    I-trigger       0.46      0.16      0.24        76
            O       0.87      0.95      0.91      6109

     acc

In [0]:
y_pred = crf.predict(X_test)
print(metrics.flat_classification_report(y_test, y_pred))

               precision    recall  f1-score   support

      B-etime       0.75      0.56      0.64        81
      B-fname       0.57      0.24      0.33        34
        B-loc       0.44      0.14      0.21        50
  B-organizer       0.67      0.39      0.50       153
B-participant       0.69      0.46      0.55       225
      B-place       0.87      0.61      0.72       116
     B-target       0.35      0.17      0.23       105
    B-trigger       0.86      0.60      0.71       309
      I-etime       0.73      0.58      0.65        96
      I-fname       0.51      0.23      0.32        86
        I-loc       0.63      0.23      0.34        82
  I-organizer       0.55      0.48      0.51       147
I-participant       0.19      0.05      0.08        62
      I-place       0.89      0.53      0.67        15
     I-target       0.27      0.26      0.26        50
    I-trigger       0.60      0.20      0.30        76
            O       0.87      0.96      0.92      6109

     acc

In [0]:
y_pred = crf.predict(X_test)
print(metrics.flat_classification_report(y_test, y_pred))

               precision    recall  f1-score   support

      B-etime       0.75      0.53      0.62        81
      B-fname       0.53      0.24      0.33        34
        B-loc       0.55      0.12      0.20        50
  B-organizer       0.66      0.39      0.49       153
B-participant       0.68      0.46      0.55       225
      B-place       0.89      0.61      0.72       116
     B-target       0.34      0.16      0.22       105
    B-trigger       0.85      0.58      0.69       309
      I-etime       0.76      0.55      0.64        96
      I-fname       0.42      0.23      0.30        86
        I-loc       0.80      0.20      0.31        82
  I-organizer       0.54      0.52      0.53       147
I-participant       0.27      0.05      0.08        62
      I-place       0.89      0.53      0.67        15
     I-target       0.28      0.26      0.27        50
    I-trigger       0.62      0.20      0.30        76
            O       0.87      0.96      0.92      6109

     acc

In [0]:
!pip install eli5

     |████████████████████████████████| 112kB 3.4MB/s 


In [0]:

import eli5
eli5.show_weights(crf, top=10)

From \ To,O,B-etime,I-etime,B-fname,I-fname,B-loc,I-loc,B-organizer,I-organizer,B-participant,I-participant,B-place,I-place,B-target,I-target,B-trigger,I-trigger
O,3.902,1.063,-2.914,1.355,-3.409,1.315,-3.118,1.644,-3.526,1.908,-2.528,1.575,-2.361,1.591,-3.344,2.242,-3.103
B-etime,0.007,-0.756,5.687,0.0,-0.054,0.0,-0.241,-0.079,0.0,-0.419,0.0,1.344,0.0,-0.145,0.0,-0.215,0.0
I-etime,0.073,-1.174,5.062,0.0,-0.258,0.0,-0.18,-0.471,-0.225,-1.356,-0.252,0.522,-0.018,-0.489,-0.344,-0.238,-0.39
B-fname,-1.131,-0.231,-0.265,0.0,5.17,0.0,-0.717,-0.154,-0.191,0.993,-0.127,-0.049,0.0,-0.394,-0.228,-0.479,0.0
I-fname,-0.293,0.305,-0.567,0.058,5.006,-1.23,-0.822,-0.103,-1.063,-0.853,-0.482,-0.66,-0.106,-0.901,-0.748,-0.301,-0.431
B-loc,0.076,0.055,-0.295,-0.279,-0.484,0.209,4.675,-0.406,-0.397,-0.537,-0.103,0.0,0.0,-0.333,-0.186,-0.052,-0.464
I-loc,0.294,-0.003,-0.433,-0.658,-0.909,-1.317,5.356,-0.483,-0.346,-0.395,0.0,-0.626,-0.017,-0.696,-0.455,-0.621,-0.473
B-organizer,-0.47,-0.359,-0.59,0.516,-0.917,-0.935,-0.591,1.477,4.203,1.172,-1.254,-0.456,-0.311,-1.584,-1.124,0.254,-0.768
I-organizer,-0.801,-0.664,-0.248,-0.947,-0.803,0.201,-0.607,1.258,4.706,0.381,-1.07,-0.63,-0.064,-1.358,-1.036,0.0,-0.383
B-participant,0.595,-0.25,-0.62,-0.546,-1.156,-0.276,-0.528,-1.384,-1.614,1.074,4.224,-0.642,-0.209,-1.593,-1.104,1.616,-1.125


In [0]:

import eli5
eli5.show_weights(crf, top=10)

From \ To,O,B-etime,I-etime,B-fname,I-fname,B-loc,I-loc,B-organizer,I-organizer,B-participant,I-participant,B-place,I-place,B-target,I-target,B-trigger,I-trigger
O,3.565,0.943,-3.258,1.05,-3.211,1.223,-3.177,2.016,-3.276,1.84,-2.408,1.524,-2.34,1.623,-3.023,2.137,-2.97
B-etime,-0.282,-0.782,5.193,-0.044,-0.125,-0.017,-0.291,0.0,0.0,-0.587,0.0,1.168,0.0,-0.35,0.0,-0.304,0.0
I-etime,0.163,-1.037,4.926,0.0,-0.12,0.0,-0.188,-0.226,-0.054,-1.07,-0.012,0.598,0.0,-0.504,-0.055,-0.044,-0.199
B-fname,-1.324,-0.324,-0.315,-0.104,5.488,-0.167,-0.615,0.0,-0.032,1.1,0.0,-0.044,0.0,-0.269,0.0,-0.647,0.0
I-fname,-0.105,0.559,-0.781,0.183,5.392,-1.229,-0.754,0.0,-0.781,-0.768,-0.471,-0.842,-0.0,-0.84,-0.526,-0.167,-0.273
B-loc,-0.075,0.058,-0.579,-0.494,-0.462,0.198,4.485,-0.226,-0.209,-0.623,0.0,0.001,0.0,-0.464,0.0,-0.126,-0.374
I-loc,0.123,0.0,-0.675,-0.684,-0.845,-1.329,5.06,-0.197,-0.247,-0.49,0.0,-0.738,-0.019,-0.801,-0.28,-0.729,-0.375
B-organizer,-0.359,-0.345,-0.592,0.426,-0.618,-0.808,-0.55,2.1,4.69,1.443,-0.941,-0.398,-0.155,-1.486,-0.771,0.461,-0.659
I-organizer,-0.739,-0.673,-0.387,-1.01,-0.642,0.262,-0.747,1.607,5.114,0.375,-0.71,-0.51,-0.005,-1.288,-0.67,0.114,-0.353
B-participant,0.843,-0.003,-0.612,-0.51,-0.911,-0.063,-0.588,-0.964,-1.402,1.246,4.739,-0.514,-0.158,-1.344,-0.604,2.016,-1.039
